
#Solving CLIQUE using SMT solvers

In this notebook, we will
look at SMT solvers and see how they can be used to formalize and solve otherwise difficult problems. Particularly, we will show how to encode CLIQUE using constraints on a set of boolean variables.


**Instructions:**
1. To get started, click on File on the top left and click "Save a copy in Drive."
This will give you an editable version of this document that you can use.
2. If you press `CMD`+`Enter` it runs the cell, and if you press `Shift`+`Enter` it runs the cell and goes to the next one.
3. Make sure you run all cells as you go through the notebook; some cells will not work properly unless the previous one
has been run too.
4. If you disconnect or are inactive for some time you should run all of the cells again.

## 0. Preliminaries (you should run this cell but there is no need to read it)

In [ ]:
!pip install z3-solver
!pip install git+https://github.com/crrivero/FormalMethodsTasting.git#subdirectory=core
from z3 import *
from tofmcore import showSolver
import networkx as nx
import itertools
from IPython.display import clear_output
clear_output()

## Encoding constraints in Z3

The goal of this notebook is to teach you about formal methods;
particularly, how you can use existing formal verification tools
(in this case, Z3) to analyze and solve your own problems.
Before we get started, let's look at some basic things we can do with Z3.

### Boolean

Suppose you have the following three boolean constraints and you want to check if there's a solution (an assignment of the variables) that satisfies all of them:

$$ x_1 \lor x_2 \lor x_3 $$

$$ \neg x_1 \implies \neg x_2$$

$$  x_1 \land x_3  $$

Let's see how we can do this using Z3.

In [ ]:
s = Solver() # initialize Z3 solver

# initialize variables

x1 = Bool('x_1') # declaring that x_1 is a boolean variable in Z3 which will be referred to as x1 in Python
x2 = Bool('x_2')
x3 = Bool('x_3')

# Note: we can also initialize multiple variables like so: x1, x2, x3 = Bools('x_1 x_2 x_3')

# we use s.add(.) to add a constraint to our solver s
# constraints can be made using many different operations such as Or, And, Not,
# equality, etc.

# here's how we would add the constraints above to our solver:

s.add( Or( x1, x2, x3 ) ) # add the first constraint
s.add( Implies( Not(x1), Not(x2) ) ) # add the second constraint
s.add( And( x1, x3 ) ) # add the third constraint

In [ ]:
# to view the constraints in our solver, we can use the following:
print( s )
# this prints the constraints as they appear in Z3 using Z3's notation

For better readability, this notebook also has a custom print function to view our constraints in LaTeX format, like so:

In [ ]:
showSolver( s )

In [ ]:
# we can use s.check() to run the solver and check whether its satisfiable:
print ( s.check() )

 "sat" means our system of constraints is satisfiable

In [ ]:
# after using s.check(),  we can use s.model() to output a solution if one exsits
solution = s.model()
print( solution )

Let's modify our system of constraint a bit and see if it's still satisfiable. Suppose we want to check if there's a solution where $x_1 = \neg x_3$. Let's see how we would do this with Z3.

In [ ]:
s.add( x1 == Not(x3) )
showSolver( s )

In [ ]:
print( s.check() ) # check if solution exists with new constraint

"unsat" means the system is not satisfiable, i.e., there is no assignment on the variables $x_1$, $x_2$, and $x_3$ that satisfies all the constraints we gave to the solver. **Note that if we were to run s.model() now we would get an error.**

We can also add arithmetic constraints to boolean variables:

$$ x_1 + x_2 + x_3 = 1 $$

To model these types of constraints, Z3 converts True/False boolean values to 1/0 integer values.

In [ ]:
s = Solver() # initialize solver

# initialize variables
x1 = Bool('x_1')
x2 = Bool('x_2')
x3 = Bool('x_3')

s.add( x1 + x2 + x3 == 1 )

print( s )

Note how Z3 uses If conditions to convert boolean variables into integers.
Our custom printer function prints these without the if conditions for better readability.

In [ ]:
showSolver( s )

In [ ]:
print( s.check() ) # check if satisfiable
print( s.model() ) # print solution

Now it's your turn! Complete the code in the cell below to find a solution satisfying the following constraints:

$$ x_1 \lor x_2 \lor x_3 $$
$$ x_1 + x_2 = x_3 $$
$$ \neg x_1 \implies x_3 $$

In [ ]:
s = Solver() # initialize solver

# initialize variables
x1 = Bool('x_1')
x2 = Bool('x_2')
x3 = True # REPLACE THIS LINE

s.add( Or( x1, x2 ) ) # REPLACE THIS LINE
s.add( x1 + x2 == 1 ) # REPLACE THIS LINE
s.add( x2 == x2 )  # REPLACE THIS LINE

showSolver( s )
print( s.check() )
print( s.model() )

Note that Z3 does not support this boolean-to-integer conversion for every operator. For example, adding the constraint `x1 >= 1` or `x1 == 1` to our solver would give an error. Proper usage dictates using integers when using arithmetic constraints, but to keep things simple for our introductory notebook we will still work with booleans.

## Graphs in Python

We'll use the graph library NetworkX to represent graphs before passing them on to Z3. Let's see some examples on how to make graphs below.

In [ ]:
# NetworkX allows one to build graphs a number of ways, and also has functions
# to construct special classes of graphs. Here's an example with a complete graph:

K4 = nx.complete_graph(4)
nx.draw(K4) # nx.draw(.) can be used to show the graph

In [ ]:
# Here's an example with a cycle graph
C7 = nx.cycle_graph(7)
nx.draw(C7)

In [ ]:
# You can define your own graphs a number of ways. One easy way is to provide a
# list of edges like so:

edgeList = [ (0,1), (0,2), (0,3), (3,4), (4,5) ]
G = nx.Graph( edgeList )
nx.draw(G)

In [ ]:
# you can get the nodes and edges of your graph like so:
print ( G.nodes() )
print ( G.edges() )

# you can read more about NetworkX's functions here: https://networkx.org/

## Making a CLIQUE solver

Recall: for a graph $G$, a $k$-clique is a subset of vertices, $S$, of $G$ such that:

1.   Each pair of vertices in $S$ is adjacent
2.   $|S| = k$

The CLIQUE problem is defined as follows:

$$ \{ \langle G, k \rangle ~|~ G \text{ is an undirected graph with a $k$-clique }  \} $$

We will build a function that, when given graph $G$ and integer $k$, outputs a Z3 system where each solution corresponds to a clique of size at least $k$. Particularly, for a graph with $n$ vertices, we will make a system with $n$ variables (one correspond to each vertex) such that if $m$ of those variables are true, where $m \geq k$, then they correspond to an $m$-clique in $G$.

In [ ]:
def makeCLIQUESolver( G, k ):
  vertices = G.nodes() # get list of vertices

  # First, we map each vertex to a unique boolean variable

  vMap = {} # a dictionary mapping each vertex to a variable

  for v in vertices:
      var_name = 'x'+str(v)
      var_v = Bool(var_name)
      vMap[v] = var_v

  s = Solver() # initialize solver

  # We will construct a system with n variables such that if k of
  # those variables are true, they correspond to a k-clique in G

  # For each pair of vertices we will add a constraint
  for u, v in itertools.combinations(vertices, 2):

    var_u = vMap[u] # get boolean variable corresponding to vertex u
    var_v = vMap[v] # get boolean variable corresponding to vertex v

    isAdjacent = (u in G.neighbors(v)) # True if u and v are adjacent

    s.add( Or( And( var_u, var_v, isAdjacent ), True ) )
    # REPLACE THE LINE ABOVE with one that ensures u and v are both selected
    # by the solver only if they are adjacent in G

  # To ensure that at least k vertices are selected, we need another constraint

  # First, let's build the following expression:
  # sum_expr = x1 + x2 + x3 + ... + xn
  # sum_expr is the expression equal to the sum of all of our variables

  sum_expr = 0
  for v in vertices:
    var_v = vMap[v]
    sum_expr += var_v

  # using sum_expr, add a constraint below that forces at least k variables to be selected

  s.add( sum_expr <= 0 ) # REPLACE THIS LINE
  return s

Let's test our solver on a few examples.

In [ ]:
G = nx.complete_graph( 4 )
s = makeCLIQUESolver( G, 3 )
nx.draw(G, with_labels=True)
print ( s.check() )
print ( s.model() )

In [ ]:
G = nx.cycle_graph( 5 )
s = makeCLIQUESolver( G, 3 )
nx.draw( G, with_labels=True )
print ( s.check() )

In [ ]:
G = nx.cycle_graph( 5 )
s = makeCLIQUESolver( G, 2 )
nx.draw( G, with_labels=True )
print ( s.check() )


### Congratulations! You just wrote an SMT solver for CLIQUE!


####If you'd like to continue your Z3 journey, you can start with this guide to learn more:
https://ericpony.github.io/z3py-tutorial/guide-examples.htm